In [ ]:
!pip -q install huggingface_hub[cli] utilsforecast praxis paxml jax[cuda12]==0.4.26 einshape


In [ ]:
!pip install -q -e git+https://github.com/google-research/timesfm.git@master#egg=timesfm

  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Installing backend dependencies ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for timesfm (pyproject.toml) ... done


In [ ]:
import timesfm
import utilsforecast

In [1]:
import yfinance as yf
import pandas as pd
import numpy as np
import datetime as dt
import warnings
warnings.filterwarnings("ignore")

In [4]:
horizon_len=30
context_len = 128+64  #we want 128 trading day's price

validation_end_date = dt.datetime.now() - dt.timedelta(days=1)
validation_start_date = validation_end_date - dt.timedelta(days=horizon_len)
end_date = validation_start_date - dt.timedelta(days=horizon_len)
start_date = end_date - dt.timedelta(days=context_len)
# timedelta has to be used before converting to string
validation_end_date = dt.datetime.strftime(validation_end_date, "%Y-%m-%d")
validation_start_date = dt.datetime.strftime(validation_start_date, "%Y-%m-%d")
start_date = dt.datetime.strftime(start_date, "%Y-%m-%d")
end_date = dt.datetime.strftime(end_date,"%Y-%m-%d")
print()
print(f"validation_end_date: {validation_end_date}")
print(f"validation_start_date: {validation_start_date}")
print(f"start_date: {start_date}")
print(f"end_date: {end_date}")


validation_end_date: 2024-06-04
validation_start_date: 2024-05-05
start_date: 2023-09-26
end_date: 2024-04-05


In [ ]:
df = yf.download(tickers='FIVESTAR.NS', start=start_date, end=end_date, interval='1d')
df["ds"] = pd.to_datetime(df.index)
df.reset_index(drop=True, inplace=True)
print(f"\nNumber of rows in the dataset: {df.shape}")
df.head(5)

[*********************100%%**********************]  1 of 1 completed


Number of rows in the dataset: (128, 7)


,Open,High,Low,Close,Adj Close,Volume,ds
0,696.549988,701.000000,688.000000,697.150024,697.150024,185521,2023-09-26
1,700.000000,700.799988,689.250000,694.150024,694.150024,214591,2023-09-27
2,695.000000,713.500000,695.000000,707.450012,707.450012,600221,2023-09-28
3,712.650024,714.700012,693.250000,696.700012,696.700012,187743,2023-09-29
4,702.299988,716.000000,696.299988,713.299988,713.299988,221710,2023-10-03


In [ ]:

tfm = timesfm.TimesFm(
    context_len=len(df),
    horizon_len=horizon_len*2,
    input_patch_len=32,
    output_patch_len=128,
    num_layers=20,
    model_dims=1280,
    backend="gpu",
)
tfm.load_from_checkpoint(repo_id="google/timesfm-1.0-200m")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

(…)nts/checkpoint_1100000/metadata/metadata:   0%|          | 0.00/43.9k [00:00<?, ?B/s]

.gitattributes:   0%|          | 0.00/1.60k [00:00<?, ?B/s]

(…)oint_1100000/descriptor/descriptor.pbtxt:   0%|          | 0.00/499 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/5.73k [00:00<?, ?B/s]

checkpoint:   0%|          | 0.00/814M [00:00<?, ?B/s]

Constructing model weights.


Constructed model weights in 4.03 seconds.
Restoring checkpoint from /root/.cache/huggingface/hub/models--google--timesfm-1.0-200m/snapshots/8775f7531211ac864b739fe776b0b255c277e2be/checkpoints.


ERROR:absl:For checkpoint version > 1.0, we require users to provide
          `train_state_unpadded_shape_dtype_struct` during checkpoint
          saving/restoring, to avoid potential silent bugs when loading
          checkpoints to incompatible unpadded shapes of TrainState.


Restored checkpoint in 1.71 seconds.
Jitting decoding.
Jitted decoding in 23.32 seconds.


In [ ]:
input_df = df[["ds","Close"]].rename(columns={"Close":"y"})
input_df["unique_id"] = input_df.index
input_df.head()

,ds,y,unique_id
0,2023-09-26,697.150024,0
1,2023-09-27,694.150024,1
2,2023-09-28,707.450012,2
3,2023-09-29,696.700012,3
4,2023-10-03,713.299988,4


In [ ]:
# import gc
# gc.collect()

223

In [ ]:
forecast_df = tfm.forecast_on_df(
    inputs=input_df,
    freq="D",  # daily
    value_name="y",
    num_jobs=-1,
)
print()
print(f"Unique values : {forecast_df['ds'].nunique()} & Shape : {forecast_df.shape}")
print(f"Start : {forecast_df['ds'].unique()[0]} & End : {forecast_df['ds'].unique()[-1]}")
forecast_df.tail()

Processing dataframe with multiple processes.
Finished preprocessing dataframe.
Finished forecasting.

Unique values : 251 & Shape : (7680, 12)
Start : 2023-09-27 00:00:00 & End : 2024-06-03 00:00:00


,unique_id,ds,timesfm,timesfm-q-0.1,timesfm-q-0.2,timesfm-q-0.3,timesfm-q-0.4,timesfm-q-0.5,timesfm-q-0.6,timesfm-q-0.7,timesfm-q-0.8,timesfm-q-0.9
7675,127,2024-05-30,753.247559,752.223999,752.575867,752.828247,753.042358,753.247559,753.452087,753.721008,754.085205,754.685059
7676,127,2024-05-31,753.259033,752.218201,752.595032,752.834167,753.055115,753.259033,753.460815,753.722351,754.070435,754.683289
7677,127,2024-06-01,753.265991,752.202759,752.580017,752.840637,753.063721,753.265991,753.461792,753.729431,754.081299,754.692627
7678,127,2024-06-02,753.293762,752.223389,752.617920,752.884094,753.090576,753.293762,753.505005,753.754456,754.104370,754.700562
7679,127,2024-06-03,753.320312,752.231934,752.624084,752.886902,753.105591,753.320312,753.526672,753.782837,754.113647,754.724182


In [ ]:
# forecast_df.to_csv("RAW-Prediction.csv",index=False)


In [2]:
raw_forecast = pd.read_csv("RAW-Prediction.csv")
print(raw_forecast.shape)
print(f"Unique values : {raw_forecast['ds'].nunique()}")
print(f"Start : {raw_forecast['ds'].unique()[0]} & End : {raw_forecast['ds'].unique()[-1]}")
raw_forecast.tail()

(7680, 12)
Unique values : 251
Start : 2023-09-27 & End : 2024-06-03


,unique_id,ds,timesfm,timesfm-q-0.1,timesfm-q-0.2,timesfm-q-0.3,timesfm-q-0.4,timesfm-q-0.5,timesfm-q-0.6,timesfm-q-0.7,timesfm-q-0.8,timesfm-q-0.9
7675,127,2024-05-30,753.24756,752.22400,752.57587,752.82825,753.04236,753.24756,753.4521,753.72100,754.08520,754.68506
7676,127,2024-05-31,753.25903,752.21820,752.59503,752.83417,753.05510,753.25903,753.4608,753.72235,754.07043,754.68330
7677,127,2024-06-01,753.26600,752.20276,752.58000,752.84064,753.06370,753.26600,753.4618,753.72943,754.08130,754.69260
7678,127,2024-06-02,753.29376,752.22340,752.61790,752.88410,753.09060,753.29376,753.5050,753.75446,754.10440,754.70056
7679,127,2024-06-03,753.32030,752.23193,752.62410,752.88690,753.10560,753.32030,753.5267,753.78284,754.11365,754.72420


In [5]:
raw_forecast = raw_forecast[(raw_forecast['ds']>validation_start_date) & (raw_forecast['ds']<validation_end_date) ]
raw_forecast = raw_forecast[['ds', 'timesfm','timesfm-q-0.1', 'timesfm-q-0.9']].rename(columns={'timesfm': 'yhat', 'timesfm-q-0.1': 'yhat_lower', 'timesfm-q-0.9': 'yhat_upper'})
raw_forecast = raw_forecast.sort_values(by=['ds','yhat','yhat_lower','yhat_upper'])
raw_forecast.tail(22)

,ds,yhat,yhat_lower,yhat_upper
7496,2024-05-28,738.45905,737.41820,739.88330
7614,2024-05-28,742.35730,741.33276,743.79700
7555,2024-05-28,746.19760,745.17400,747.63510
7673,2024-05-28,753.25226,752.24365,754.68744
7497,2024-05-29,738.46600,737.40280,739.89264
7615,2024-05-29,742.34753,741.32400,743.78503
7556,2024-05-29,746.20905,745.16820,747.63330
7674,2024-05-29,753.25730,752.23280,754.69700
7498,2024-05-30,738.49380,737.42340,739.90060
7616,2024-05-30,742.35900,741.31820,743.78326


In [6]:
# if single prediction then confidence is high
# but for multiple prediction confidence is low

ds_prediction = []
n_bootstraps = 1000  #
confidence_values = []
for date in raw_forecast['ds'].unique():
    date_data = raw_forecast[raw_forecast['ds'] == date]
    num_predictions = len(date_data)
    ds_prediction.append([date_data,num_predictions])
    print(f"For {date} no. of predictions : {num_predictions}")
    bootstrapped_predictions = np.random.choice(date_data.iloc[0, 1:num_predictions+1], size=(n_bootstraps, num_predictions))
    confidence = np.mean((date_data.iloc[0]['yhat'] >= bootstrapped_predictions.min(axis=1)) &
                        (date_data.iloc[0]['yhat'] <= bootstrapped_predictions.max(axis=1)))
    confidence_values.append({'ds': date, 'confidence': confidence})

confidence_df = pd.DataFrame(confidence_values)
raw_forecast = pd.merge(raw_forecast, confidence_df, on='ds')
print()
print(f"No. of predictions(days) : {len(ds_prediction)}")

For 2024-05-06 no. of predictions : 18
For 2024-05-07 no. of predictions : 17
For 2024-05-08 no. of predictions : 17
For 2024-05-09 no. of predictions : 17
For 2024-05-10 no. of predictions : 17
For 2024-05-11 no. of predictions : 16
For 2024-05-12 no. of predictions : 15
For 2024-05-13 no. of predictions : 14
For 2024-05-14 no. of predictions : 13
For 2024-05-15 no. of predictions : 12
For 2024-05-16 no. of predictions : 12
For 2024-05-17 no. of predictions : 12
For 2024-05-18 no. of predictions : 11
For 2024-05-19 no. of predictions : 10
For 2024-05-20 no. of predictions : 9
For 2024-05-21 no. of predictions : 8
For 2024-05-22 no. of predictions : 7
For 2024-05-23 no. of predictions : 7
For 2024-05-24 no. of predictions : 7
For 2024-05-25 no. of predictions : 7
For 2024-05-26 no. of predictions : 6
For 2024-05-27 no. of predictions : 5
For 2024-05-28 no. of predictions : 4
For 2024-05-29 no. of predictions : 4
For 2024-05-30 no. of predictions : 4
For 2024-05-31 no. of predictions : 

In [7]:
# date - ascending ,confidence - descending : 1st highest confidence
raw_forecast_sorted = raw_forecast.sort_values(['ds', 'confidence'], ascending=[True, False])
raw_forecast_sorted.tail(22)

,ds,yhat,yhat_lower,yhat_upper,confidence
257,2024-05-28,738.45905,737.41820,739.88330,0.974
258,2024-05-28,742.35730,741.33276,743.79700,0.974
259,2024-05-28,746.19760,745.17400,747.63510,0.974
260,2024-05-28,753.25226,752.24365,754.68744,0.974
261,2024-05-29,738.46600,737.40280,739.89264,0.969
262,2024-05-29,742.34753,741.32400,743.78503,0.969
263,2024-05-29,746.20905,745.16820,747.63330,0.969
264,2024-05-29,753.25730,752.23280,754.69700,0.969
265,2024-05-30,738.49380,737.42340,739.90060,0.969
266,2024-05-30,742.35900,741.31820,743.78326,0.969


In [8]:
# we take min,median & max prediction value based on 90% quantile prediction
result = raw_forecast_sorted.groupby('ds')['yhat_upper'].agg(['min', 'median', 'max']).reset_index()
result.columns = ['ds', 'yhat_min', 'yhat_mode', 'yhat_max']

raw_forecast_filtered = pd.merge(raw_forecast_sorted, result, on='ds', how='left')
raw_forecast_filtered

,ds,yhat,yhat_lower,yhat_upper,confidence,yhat_min,yhat_mode,yhat_max
0,2024-05-06,609.70230,608.69366,611.13745,1.000,611.13745,689.704435,754.82410
1,2024-05-06,616.95030,615.93494,618.34420,1.000,611.13745,689.704435,754.82410
2,2024-05-06,625.78880,624.76220,627.20260,1.000,611.13745,689.704435,754.82410
3,2024-05-06,642.77124,641.83120,643.99890,1.000,611.13745,689.704435,754.82410
4,2024-05-06,652.34590,651.40344,653.53876,1.000,611.13745,689.704435,754.82410
...,...,...,...,...,...,...,...,...
274,2024-06-01,746.27030,745.18195,747.67420,0.923,743.80054,747.674200,754.69260
275,2024-06-01,753.26600,752.20276,754.69260,0.923,743.80054,747.674200,754.69260
276,2024-06-02,742.42030,741.33190,743.82416,0.750,743.82416,749.262360,754.70056
277,2024-06-02,753.29376,752.22340,754.70056,0.750,743.82416,749.262360,754.70056


In [25]:
df_predicted = raw_forecast_filtered.drop_duplicates(subset='ds',keep='first').reset_index(drop=True)
df_predicted["ds"] = pd.to_datetime(df_predicted["ds"])
df_predicted = df_predicted[["ds","yhat_min","yhat_mode","yhat_max","confidence"]].rename(columns={"yhat_mode":"Predicted_Close"})
df_predicted

,ds,yhat_min,Predicted_Close,yhat_max,confidence
0,2024-05-06,611.13745,689.704435,754.82410,1.000
1,2024-05-07,611.14703,699.131800,754.79750,1.000
2,2024-05-08,611.13510,699.115300,754.77356,1.000
3,2024-05-09,611.13330,699.120600,754.78894,1.000
4,2024-05-10,611.14264,699.104600,754.76390,1.000
5,2024-05-11,611.15060,699.945005,754.78864,1.000
6,2024-05-12,611.17420,700.894200,754.79280,1.000
7,2024-05-13,618.35050,708.948400,754.82935,1.000
8,2024-05-14,618.37415,717.002560,754.82104,1.000
9,2024-05-15,644.09260,718.788075,754.83970,1.000


In [10]:
original_df = yf.download(tickers='FIVESTAR.NS', start=validation_start_date, end=validation_end_date, interval='1d')

original_df["ds"] = pd.to_datetime(original_df.index)
original_df.reset_index(drop=True, inplace=True)
original_df = original_df[["ds","Open","Close","High","Low"]]
original_df


[*********************100%%**********************]  1 of 1 completed


,ds,Open,Close,High,Low
0,2024-05-06,780.200012,781.150024,790.000000,766.049988
1,2024-05-07,780.549988,771.400024,783.750000,751.099976
2,2024-05-08,775.000000,750.049988,775.799988,741.200012
3,2024-05-09,751.000000,717.700012,758.700012,710.000000
4,2024-05-10,726.000000,728.849976,733.000000,706.549988
5,2024-05-13,732.799988,722.000000,749.599976,715.200012
6,2024-05-14,722.000000,748.099976,753.900024,721.000000
7,2024-05-15,756.049988,767.849976,770.500000,742.849976
8,2024-05-16,760.900024,741.799988,763.799988,738.599976
9,2024-05-17,754.950012,739.250000,754.950012,735.000000


In [29]:
merged_df = pd.merge(original_df,df_predicted[["ds","Predicted_Close","confidence"]] , on='ds', how='left')
merged_df

,ds,Open,Close,High,Low,Predicted_Close,confidence
0,2024-05-06,780.200012,781.150024,790.000000,766.049988,689.704435,1.000
1,2024-05-07,780.549988,771.400024,783.750000,751.099976,699.131800,1.000
2,2024-05-08,775.000000,750.049988,775.799988,741.200012,699.115300,1.000
3,2024-05-09,751.000000,717.700012,758.700012,710.000000,699.120600,1.000
4,2024-05-10,726.000000,728.849976,733.000000,706.549988,699.104600,1.000
5,2024-05-13,732.799988,722.000000,749.599976,715.200012,708.948400,1.000
6,2024-05-14,722.000000,748.099976,753.900024,721.000000,717.002560,1.000
7,2024-05-15,756.049988,767.849976,770.500000,742.849976,718.788075,1.000
8,2024-05-16,760.900024,741.799988,763.799988,738.599976,718.797970,1.000
9,2024-05-17,754.950012,739.250000,754.950012,735.000000,718.801275,1.000


In [31]:
import plotly.graph_objects as go


fig = go.Figure(data=[go.Candlestick(x=merged_df['ds'],open=merged_df['Open'], high=merged_df['High'],
                low=merged_df['Low'], close=merged_df['Close'])])

fig.add_trace(go.Scatter(x=merged_df['ds'], y=merged_df['Predicted_Close'],
                         fill=None, mode='lines', line_color='black', name='Predicted_Close'))

fig.update_layout(title='OHLC Chart with Predicted_Close',xaxis_title='Date',yaxis_title='Price')

fig.show()

In [ ]:
#